In [1]:
import pandas as pd

import numpy as np

import pymongo

from slugify import slugify

/home/johni/.pyenv/versions/3.7.4/envs/iara-ws-cnmp/lib/python3.7/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
MONGODB_URL = "mongodb://35.237.232.73:27017/"

client = pymongo.MongoClient(MONGODB_URL)

In [3]:
DATABASE_NAME = "dados_diversos"

COLLECTION_NAME = "mpm_serventias"

serventias = list(client[DATABASE_NAME][COLLECTION_NAME].find())

print(f"Total de seventias {len(serventias)}")

Total de seventias 39099


In [4]:
class Base:
    def __init__(self, filename, csv=False):
        if csv:
            self.df = pd.read_csv(filename, sep=";")
        else:
            self.df = pd.read_excel(filename, sep=";")

    @property
    def shape(self):
        return self.df.shape
    
    def head(self, n=5):
        return self.df.head(n)

# Entrância


In [5]:
class Entrancia(Base):
    def __init__(self):
        Base.__init__(self, "mpm_serventias.csv", True)
    
    def processar(self, documents):
        for documento in documents:
            seq_orgao = int(documento["SEQ_ORGAO"])

            entrancia = self._calcular(seq_orgao)

            self._salvar(documento, entrancia)
    
    def _calcular(self, seq_orgao): 
        df_f_orgao = self.df[self.df["SEQ_ORGAO"] == seq_orgao]

        if len(df_f_orgao) == 0:
            print(f"Orgão {seq_orgao} não encontrado!")
            return None

        seq_orgao_pai = df_f_orgao['SEQ_ORGAO_PAI'].values[0]

        """
        Filtra as varas que tem o mesmo orgão pai
        """
        df_f_orgao_pai = self.df[self.df["SEQ_ORGAO_PAI"] == seq_orgao_pai]

        if len(df_f_orgao_pai) == 0:
            print(f"Orgão pai {seq_orgao_pai} não encontrado para o orgão {seq_orgao}!")
            return None

        total = df_f_orgao_pai["SEQ_ORGAO_PAI"].notnull().count()

        if total == None:
            print(f"Orgao {seq_orgao} não possui orgão pai!")
            return None

        if total == 1:
            return 1
        elif total in [2, 3, 4]:
            return 2
        else:
            return 3
    
    def _salvar(self, documento, entrancia):

        collection = client[DATABASE_NAME][COLLECTION_NAME]

        _id =  documento["_id"]

        collection.update_one(
            {
                "_id": _id 
            }, 
            {
                '$set': { 'TIP_ENTRANCIA': entrancia }
            }
        )

In [6]:
et = Entrancia()
et.shape

/home/johni/.pyenv/versions/3.7.4/envs/iara-ws-cnmp/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3331: DtypeWarning: Columns (12,14) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


(39100, 17)

In [7]:
et.head()

,SEQ_ORGAO,NOMEDAVARA,SEQ_ORGAO_PAI,TIP_ORGAO,SEQ_CIDADE,DSC_CIDADE,SIG_UF,COD_IBGE,DSC_TIP_ORGAO,TIP_ESFERA_JUSTICA,INT_ORDEM_ORGAO,DSC_ORGAO,DSC_DENOM_SERVENTIA_JUDICIAL,endereco_serventia,cep_serventia,LATITUDE,LONGITUDE
0,1,Conselho Nacional de Justiça,NaN,CNJ,743.0,BRASÍLIA,DF,5300108.0,Conselho Nacional de Justiça,S,0.0,Conselho Nacional de Justiça,NaN,NaN,NaN,NaN,NaN
1,2,Conselho da Justiça Federal,1.0,CONSF,743.0,BRASÍLIA,DF,5300108.0,Conselho da Justiça Federal,F,0.0,Conselho da Justiça Federal,NaN,NaN,NaN,NaN,NaN
2,3,Tribunal Regional Federal da 1ª Região,2.0,TRIBF,743.0,BRASÍLIA,DF,5300108.0,Tribunal Regional Federal,F,NaN,Tribunal Regional Federal da 1ª Região,NaN,NaN,NaN,NaN,NaN
3,4,Tribunal Regional Federal da 2ª Região,2.0,TRIBF,4155.0,RIO DE JANEIRO,RJ,3304557.0,Tribunal Regional Federal,F,NaN,Tribunal Regional Federal da 2ª Região,NaN,NaN,NaN,NaN,NaN
4,5,Tribunal Regional Federal da 3ª Região,2.0,TRIBF,4795.0,SAO PAULO,SP,3550308.0,Tribunal Regional Federal,F,NaN,Tribunal Regional Federal da 3ª Região,NaN,NaN,NaN,1050182067,607162856


In [ ]:
et.processar(serventias)

# Força de Trabalho

In [ ]:
class ForcaDeTrabalho(Base):
    def __init__(self):
        Base.__init__(self, "dados-pessoal-TJSP.xlsx")
        
    def processar(self, documentos):
        for documento in documentos:
            seq_orgao = int(documento["SEQ_ORGAO"])
            
            forca_trabalho = self._buscar(seq_orgao)
            self._inseir(documento, forca_trabalho)

    def _inseir(self, documento, forca_trabalho):
        collection = client[DATABASE_NAME][COLLECTION_NAME]
    
        _id = documento["_id"]

        collection.update_one(
            {
                "_id": _id 
            },
            {
                "$set": { 
                    "QTD_ESTAGIARIOS": forca_trabalho["qtd_estagiarios"],
                    "QTD_TERCERIADOS": forca_trabalho["qtd_terceriados"],
                    "QTD_CEDIDOS": forca_trabalho["qtd_cedidos"],
                    "QTD_COMISSIONADO": forca_trabalho["qtd_comissionado"],
                    "QTD_EFETIVOS": forca_trabalho["qtd_efetivos"],
                    
                }
            }
        )

    def _buscar(self, seq_orgao):
        forca_trabalho = {
            "qtd_estagiarios": None,
            "qtd_terceriados": None,
            "qtd_cedidos": None,
            "qtd_comissionado": None,
            "qtd_efetivos": None,
        }

        df_f = self.df[self.df["seq_orgao"] == seq_orgao]

        if len(df_f) == 0:
            return forca_trabalho

        for _, row in df_f.iterrows():
            
            descricao = slugify(row["Descrição da variável"])
            quantidade = int(row["Qtd"])
            
            if descricao == "tfauxe-total-da-forca-de-trabalho-auxiliar-estagiarios":
                forca_trabalho["qtd_estagiarios"] = quantidade 
                
            if descricao == "tfauxt-total-da-forca-de-trabalho-auxiliar-terceirizados":
                forca_trabalho["qtd_terceriados"] = quantidade
                
            if descricao == "tpefet-total-de-pessoal-do-quadro-efetivo":
                forca_trabalho["qtd_efetivos"] = quantidade
                
            if descricao == "tpi-total-de-pessoal-que-ingressou-por-cessao-ou-requisicao":
                forca_trabalho["qtd_cedidos"] = quantidade
                
            if descricao == "tpsv-total-de-pessoal-comissionado-sem-vinculo":
                forca_trabalho["qtd_comissionado"] = quantidade
            
        return forca_trabalho

In [ ]:
ft = ForcaDeTrabalho()
ft.shape

In [ ]:
ft.head()

In [ ]:
ft.processar(serventias)

# Pocessos Pendentes

In [ ]:
class ProcessosPendentes(Base):
    def __init__(self):
        Base.__init__(self, "casos-pendentes-TJSP.xlsx")
    
    def processar(self, documentos):
        for documento in documentos:
            seq_orgao = int(documento["SEQ_ORGAO"])

            processos_pendentes = self._buscar(seq_orgao)
            self._inseir(documento, processos_pendentes)
        
    def _inseir(self, documento, processos_pendentes):
        collection = client[DATABASE_NAME][COLLECTION_NAME]

        _id = documento["_id"]

        collection.update_one(
            {
                "_id": _id 
            },
            {
                '$set': { 'QTD_PROC_PENDENTES': processos_pendentes }
            }
        )

    def _buscar(self, seq_orgao):
        df_f = self.df[self.df["seq_orgao"] == seq_orgao]

        if len(df_f) != 1:
            return None
        
        processos_pendentes = int(df_f['Qtd'].values[0])
        
        return processos_pendentes        

In [ ]:
pp = ProcessosPendentes()
pp.shape

In [ ]:
pp.head()

In [ ]:
pp.processar(serventias)